# Exercise 03: CLIP zero-shot prediction
In this exercise, you will perform zero-shot prediction using CLIP.

### Basic Imports

In [1]:
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision.datasets import CIFAR10
from torchvision import datasets
from torchvision import transforms
import torchvision

from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
from PIL import Image
from clip import clip

### Hyperparameters

In [2]:
# # random seed
# SEED = 1 
# NUM_CLASS = 10

# Training
BATCH_SIZE = 128
# NUM_EPOCHS = 30
# EVAL_INTERVAL=1
# SAVE_DIR = './log'

# # Optimizer
# LEARNING_RATE = 1e-1
# MOMENTUM = 0.9
# STEP=5
# GAMMA=0.5

# CLIP
VISUAL_BACKBONE = 'RN50' # RN50, ViT-B/32, ViT-B/16


### Device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### Dataset


In [4]:
transform_cifar10_test = transforms.Compose([
    transforms.Resize(size=224),
    transforms.CenterCrop(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

test_set = torchvision.datasets.CIFAR10(root='/shareddata', train=False,
                                       download=True, transform=transform_cifar10_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
dataset_name = 'CIFAR10'

Files already downloaded and verified


### Model

In [13]:
# Load the model
model, preprocess = clip.load(name=VISUAL_BACKBONE, device=device, download_root='/shareddata/clip/')
model.to(device)
1

1

### Task 1: Prompt Gereration
---

Please denfine a function named ``prompt_encode`` to encode the text using CLIP text encoder.


In [6]:
prompt = 'a photo of a' # you can try different prompt

def prompt_encode(prompt):
    """
    Args:
        prompt (str): the text prefix before the class

    Returns:
        text_inputs(torch.Tensor)

    """
    ##################### Write your answer here ##################
    # 将文本编码为tensor    
    text_inputs= torch.cat([clip.tokenize(f"{prompt} {c}") for c in class_names]).to(device)
    ###############################################################
    return text_inputs

# 编码文本提示
text_inputs = prompt_encode(prompt).to(device)

### Task 2: Zero-shot inference
---

Please denfine a function named ``model_inference``. The function is essential for training and evaluating machine learning models using batched data from dataloaders.

**To do**: 
1. Encode the image.
2. Encode the text.
3. Calculate the logits.

In [7]:
def model_inference(model, image, text_inputs):
    """
    Args:
        model (torch.nn.Module): 训练好的机器学习模型
        image (torch.Tensor): 输入图像张量
        text_inputs (torch.Tensor): 输入文本张量

    Returns:
        logits (torch.Tensor): 模型的预测logits
    """
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_inputs)

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    logits = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    
    return logits

### Task 3: Zero-shot accuracy calculation
---

#### RN50

In [12]:
testing_loss = []
testing_acc = []

with torch.no_grad():
    model.eval()  
    ##################### Write your answer here ##################
    correct_predictions = 0
    for image, class_id in test_dataloader:
        image=image.to(device)
        class_id=class_id.to(device)
        
        logits=model_inference(model,image,text_inputs)
        
        predictions = logits.argmax(dim=-1)
        correct_predictions += (predictions == class_id).sum().item()
    
    val_acc = correct_predictions / len(test_set)
    
    ###############################################################

    print(f"the zero-shot performance on {dataset_name} is {val_acc*100:.2f}%, visual encoder is {VISUAL_BACKBONE}.")

the zero-shot performance on CIFAR10 is 55.84%, visual encoder is RN50.


#### VIT-B/32

In [15]:
VISUAL_BACKBONE = 'ViT-B/32' # RN50, ViT-B/32, ViT-B/16
model, preprocess = clip.load(name=VISUAL_BACKBONE, device=device, download_root='/shareddata/clip/')
testing_loss = []
testing_acc = []

with torch.no_grad():
    model.eval()  
    ##################### Write your answer here ##################
    correct_predictions = 0
    for image, class_id in test_dataloader:
        image=image.to(device)
        class_id=class_id.to(device)
        
        logits=model_inference(model,image,text_inputs)
        
        predictions = logits.argmax(dim=-1)
        correct_predictions += (predictions == class_id).sum().item()
    
    val_acc = correct_predictions / len(test_set)
    
    ###############################################################

    print(f"the zero-shot performance on {dataset_name} is {val_acc*100:.2f}%, visual encoder is {VISUAL_BACKBONE}.")

the zero-shot performance on CIFAR10 is 85.47%, visual encoder is ViT-B/32.


#### ViT-B/16

In [19]:
VISUAL_BACKBONE = 'ViT-B/16' 
model, preprocess = clip.load(name=VISUAL_BACKBONE, device=device, download_root='/shareddata/clip/')
testing_loss = []
testing_acc = []

with torch.no_grad():
    model.eval()  
    ##################### Write your answer here ##################
    correct_predictions = 0
    for image, class_id in test_dataloader:
        image=image.to(device)
        class_id=class_id.to(device)
        
        logits=model_inference(model,image,text_inputs)
        
        predictions = logits.argmax(dim=-1)
        correct_predictions += (predictions == class_id).sum().item()
    
    val_acc = correct_predictions / len(test_set)
    
    ###############################################################

    print(f"the zero-shot performance on {dataset_name} is {val_acc*100:.2f}%, visual encoder is {VISUAL_BACKBONE}.")

the zero-shot performance on CIFAR10 is 88.03%, visual encoder is ViT-B/16.
